<a href="https://colab.research.google.com/github/Jamie-Huang/NBA-Prediction-Model/blob/main/Master_NBA_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bs4
!pip install urlopen
from urllib.request import urlopen
from google.colab import files
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

from google.colab import drive 
import tensorflow as tf
from tensorflow import keras
import keras
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense
drive.mount('/content/drive')

  Created wheel for urlopen: filename=urlopen-1.0.0-py3-none-any.whl size=1409 sha256=0d8b5d1d3512add20df1438f8f6f2ce7c22dc6e67cca99eea724194e1357e930
  Stored in directory: /root/.cache/pip/wheels/03/ef/31/96a0d9f1487a5af8b393e88ee04994b20ba0b988c922e3aabd
Successfully built urlopen
Mounted at /content/drive


**GET PLAYER DATA FROM 1990 - 2000**

In [ ]:
r_df = pd.DataFrame()
ps_df = pd.DataFrame()
years = list(range(1990,2001))
seasons = ['Ps','Reg']
per100url = 'https://www.basketball-reference.com/leagues/NBA_{}_per_poss.html'
psper100url = 'https://www.basketball-reference.com/playoffs/NBA_{}_per_poss.html'


advurl = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'
psadvurl = 'https://www.basketball-reference.com/playoffs/NBA_{}_advanced.html'

pergmeurl = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'
pspergmeurl = 'https://www.basketball-reference.com/playoffs/NBA_{}_per_game.html'

toturl = 'https://www.basketball-reference.com/leagues/NBA_{}_totals.html'
pstoturl = 'https://www.basketball-reference.com/playoffs/NBA_{}_totals.html'

for year in years:
  for season in seasons:
    if season == 'Ps':
      ##per 100 stats
      p100_url = psper100url.format(year)
      # collect HTML data
      p100_html = urlopen(p100_url)
              
      # create beautiful soup object from HTML
      p100_soup = BeautifulSoup(p100_html, features="lxml")
      p100_headers = [th.getText() for th in p100_soup.findAll('tr', limit=2)[0].findAll('th')]
      p100_headers = p100_headers[1:]
      #headers = list(filter(None,headers))

      ps_headers = p100_headers[4:]
      ps_headers = list(map(lambda x: 'Ps' + x, ps_headers))
      p100_headers = p100_headers[0:4] + ps_headers

      p100_rows = p100_soup.findAll('tr')[1:]
      p100_rows_data = [[td.getText() for td in p100_rows[i].findAll('td')]for i in range(len(p100_rows))]
      ps_player_stats = pd.DataFrame(p100_rows_data,columns=p100_headers)
      ps_player_stats['Year'] = year
      

      ##advanced stats
      adv_url = psadvurl.format(year)
      adv_html = urlopen(adv_url)

      adv_soup = BeautifulSoup(adv_html, features="lxml")
      adv_headers = [th.getText() for th in adv_soup.findAll('tr', limit=2)[0].findAll('th')]
      adv_headers = adv_headers[1:]
    
      ps_adv_headers = adv_headers[4:]
      ps_adv_headers = list(map(lambda x: 'Ps' + x, ps_adv_headers))
      adv_headers = adv_headers[0:4] + ps_adv_headers
        

      adv_rows = adv_soup.findAll('tr')[1:]
      adv_rows_data = [[td.getText() for td in adv_rows[i].findAll('td')]for i in range(len(adv_rows))]
      adv_ps_player_stats = pd.DataFrame(adv_rows_data,columns=adv_headers)

      
      ##totals
      tot_url = pstoturl.format(year)
      # collect HTML data
      tot_html = urlopen(tot_url)
              
      # create beautiful soup object from HTML
      tot_soup = BeautifulSoup(tot_html, features="lxml")
      tot_headers = [th.getText() for th in tot_soup.findAll('tr', limit=2)[0].findAll('th')]
      tot_headers = tot_headers[1:]
      #headers = list(filter(None,headers))

      ps_tot_headers = tot_headers[4:]
      ps_tot_headers = list(map(lambda x: 'Ps' + x, ps_tot_headers))
      tot_headers = tot_headers[0:4] + ps_tot_headers

      tot_rows = tot_soup.findAll('tr')[1:]
      tot_rows_data = [[td.getText() for td in tot_rows[i].findAll('td')]for i in range(len(tot_rows))]
      ps_tot_player_stats = pd.DataFrame(tot_rows_data,columns=tot_headers)
      ps_tot_player_stats.insert(3,'Year',season + str(year))

      ps_tot_player_stats.replace('',0,inplace=True)
      ps_tot_player_stats = pd.concat([ps_tot_player_stats.iloc[:,:4].copy(), ps_tot_player_stats.iloc[:,5:].astype('float64').copy()], axis=1)
      
      ps_pts = ps_tot_player_stats['PsPTS'].sum()
      ps_fga = ps_tot_player_stats['PsFGA'].sum()
      ps_fta = ps_tot_player_stats['PsFTA'].sum()
      ps_ts = ps_pts/(2 * (ps_fga + 0.44 * ps_fta))

      #index certain columns
      adv_ps_player_stats = adv_ps_player_stats[['Player','Pos','Age','Tm','PsTS%','PsTRB%','PsSTL%','PsBLK%']]


      #join
      ps_player_stats = ps_player_stats.merge(adv_ps_player_stats,on=['Player','Pos','Age','Tm'])

      ps_player_stats = ps_player_stats.loc[:,ps_player_stats.columns != 'Ps']
      ps_player_stats.replace('',0,inplace=True)

      ps_player_stats = pd.concat([ps_player_stats.iloc[:,:4].copy(), ps_player_stats.iloc[:,5:].astype('float64').copy()], axis=1)

      #OC insertion
      prof = (2/(1 + np.exp(- (ps_player_stats['Ps3PA']))) - 1) * ps_player_stats['Ps3P%']

      ps_player_stats['PsOffensive Creation'] = ps_player_stats['PsAST'] * 0.1843 + (ps_player_stats['PsPTS'] + ps_player_stats['PsTOV']) * 0.0969 - 2.3021 * prof + 0.0582 * (ps_player_stats['PsAST'] * (ps_player_stats['PsTOV'] + ps_player_stats['PsPTS']) * prof) - 1.1942
      # ps_player_stats['PsAST'] * 0.1843 + (ps_player_stats['PsPTS'] + ps_player_stats['PsTOV']) * 0.0969 - 2.3021*(2/(1+np.exp(- (ps_player_stats['Ps3PA'])))-1)*ps_player_stats['Ps3P%'] + 0.0582 * (ps_player_stats['PsAST'] * (ps_player_stats['PsTOV'] + ps_player_stats['PsPTS'] * (2/(1+ np.exp(-(ps_player_stats['Ps3PA']))-1)*ps_player_stats['Ps3P%'])))
      ps_player_stats['PsrTS%'] = ps_player_stats['PsTS%'] - ps_ts

      ps_player_stats['PsOffensive Load'] = (ps_player_stats['PsAST'] - (0.38 * ps_player_stats['PsOffensive Creation']) * 0.75) + ps_player_stats['PsFGA'] + 0.44 * ps_player_stats['PsFTA'] + ps_player_stats['PsOffensive Creation'] + ps_player_stats['PsTOV']

    else:
      ##per 100 stats
      p100_url = per100url.format(year)
      # collect HTML data
      p100_html = urlopen(p100_url)
              
      # create beautiful soup object from HTML
      p100_soup = BeautifulSoup(p100_html, features="lxml")
      p100_headers = [th.getText() for th in p100_soup.findAll('tr', limit=2)[0].findAll('th')]
      p100_headers = p100_headers[1:]


      p100_rows = p100_soup.findAll('tr')[1:]
      p100_rows_data = [[td.getText() for td in p100_rows[i].findAll('td')]for i in range(len(p100_rows))]
      reg_player_stats = pd.DataFrame(p100_rows_data,columns=p100_headers)
      reg_player_stats['Year'] = year
      
      print(reg_player_stats)
      ##advanced stats
      adv_url = advurl.format(year)
      adv_html = urlopen(adv_url)

      adv_soup = BeautifulSoup(adv_html, features="lxml")
      adv_headers = [th.getText() for th in adv_soup.findAll('tr', limit=2)[0].findAll('th')]
      adv_headers = adv_headers[1:]


      adv_rows = adv_soup.findAll('tr')[1:]
      adv_rows_data = [[td.getText() for td in adv_rows[i].findAll('td')]for i in range(len(adv_rows))]
      adv_reg_player_stats = pd.DataFrame(adv_rows_data,columns=adv_headers)

      
      ##totals
      tot_url = toturl.format(year)
      # collect HTML data
      tot_html = urlopen(tot_url)
              
      # create beautiful soup object from HTML
      tot_soup = BeautifulSoup(tot_html, features="lxml")
      tot_headers = [th.getText() for th in tot_soup.findAll('tr', limit=2)[0].findAll('th')]
      tot_headers = tot_headers[1:]


      tot_rows = tot_soup.findAll('tr')[1:]
      tot_rows_data = [[td.getText() for td in tot_rows[i].findAll('td')]for i in range(len(tot_rows))]
      tot_player_stats = pd.DataFrame(tot_rows_data,columns=tot_headers)
      tot_player_stats.insert(3,'Year',season + str(year))

      tot_player_stats.replace('',0,inplace=True)
      tot_player_stats = pd.concat([tot_player_stats.iloc[:,:4].copy(), tot_player_stats.iloc[:,5:].astype('float64').copy()], axis=1)
      
      reg_pts = tot_player_stats['PTS'].sum()
      reg_fga = tot_player_stats['FGA'].sum()
      reg_fta = tot_player_stats['FTA'].sum()
      reg_ts = reg_pts/(2 * (reg_fga + 0.44 * reg_fta))

      #index certain columns
      adv_reg_player_stats = adv_reg_player_stats[['Player','Pos','Age','Tm','TS%','TRB%','STL%','BLK%']]


      #join
      
      reg_player_stats = reg_player_stats.merge(adv_reg_player_stats,on=['Player','Pos','Age','Tm'])
      reg_player_stats.replace('',0,inplace=True)

      reg_player_stats = reg_player_stats.loc[:,reg_player_stats.columns != '']

      reg_player_stats = pd.concat([reg_player_stats.iloc[:,:4].copy(), reg_player_stats.iloc[:,5:].astype('float64').copy()], axis=1)

      #OC insertion
      prof = (2/(1 + np.exp(- (reg_player_stats['3PA']))) - 1) * reg_player_stats['3P%']

      reg_player_stats['Offensive Creation'] = reg_player_stats['AST'] * 0.1843 + (reg_player_stats['PTS'] + reg_player_stats['TOV']) * 0.0969 - 2.3021 * prof + 0.0582 * (reg_player_stats['AST'] * (reg_player_stats['TOV'] + reg_player_stats['PTS']) * prof) - 1.1942
      # ps_player_stats['PsAST'] * 0.1843 + (ps_player_stats['PsPTS'] + ps_player_stats['PsTOV']) * 0.0969 - 2.3021*(2/(1+np.exp(- (ps_player_stats['Ps3PA'])))-1)*ps_player_stats['Ps3P%'] + 0.0582 * (ps_player_stats['PsAST'] * (ps_player_stats['PsTOV'] + ps_player_stats['PsPTS'] * (2/(1+ np.exp(-(ps_player_stats['Ps3PA']))-1)*ps_player_stats['Ps3P%'])))
      reg_player_stats['rTS%'] = reg_player_stats['TS%'] - reg_ts
      print(ps_player_stats)

      reg_player_stats['Offensive Load'] = (reg_player_stats['AST'] - (0.38 * reg_player_stats['Offensive Creation']) * 0.75) + reg_player_stats['FGA'] + 0.44 * reg_player_stats['FTA'] + reg_player_stats['Offensive Creation'] + reg_player_stats['TOV']

  ps_df = ps_df.append(ps_player_stats)
  r_df = r_df.append(reg_player_stats)
  #   temp_df = reg_player_stats.join(ps_player_stats, on = ['Player','Pos','Age','Tm'])
  #   df = df.append(temp_df)
  # # df = df.append(reg_player_stats)

  
  
  

               Player   Pos   Age    Tm     G  ...   PTS        ORtg  DRtg  Year
0          Mark Acres     C    27   ORL    80  ...   9.9         110   114  1990
1       Michael Adams    PG    27   DEN    79  ...  20.7         114   109  1990
2        Mark Aguirre    SF    30   DET    78  ...  27.9         111   106  1990
3         Danny Ainge    PG    30   SAC    75  ...  24.4         107   110  1990
4         Mark Alarie    PF    26   WSB    82  ...  21.9         106   110  1990
..                ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...
473  Haywoode Workman    PG    24   ATL     6  ...  18.8         175    94  1990
474     James Worthy*    SF    28   LAL    80  ...  28.4         119   108  1990
475       Danny Young    PG    27   POR    82  ...  12.9         109   105  1990
476              None  None  None  None  None  ...  None  None  None  None  1990
477     Michael Young    SF    29   LAC    45  ...  23.1         111   107  1990

[478 rows x 32 columns]
   

In [ ]:
df = pd.merge(r_df,ps_df,on = ['Player','Pos','Age','Tm','Year'],how = 'outer')
df.replace(np.nan, 0, inplace=True)
pd.set_option('display.float_format', lambda x: f'%.{len(str(x%1))-2}f' % x)

df.Year = df.Year.astype('int64',copy = True)
df.GS = df.GS.astype('int64',copy = True)
df.MP = df.MP.astype('int64',copy = True)

df.PsGS = df.PsGS.astype('int64',copy = True)
df.PsMP = df.PsMP.astype('int64',copy = True)
#pd.set_option('display.float_format','{:.2f}')
df['Offensive Creation'] = df['Offensive Creation'].astype('float').round(3)
df['Offensive Load'] = df['Offensive Load'].astype('float').round(3)
df['rTS%'] = df['rTS%'].astype('float').round(3)
df['PsOffensive Creation'] = df['PsOffensive Creation'].astype('float').round(3)
df['PsOffensive Load'] = df['PsOffensive Load'].astype('float').round(3)
df['PsrTS%'] = df['PsrTS%'].astype('float').round(3)

df = df[df['Player'] != 0]
cols = df.columns.tolist()
cols.remove('Year')
cols = cols[:2] + ['Year'] + cols[3:]
df = df[cols]

# def rep(x):
#   return x.replace('*','')
df['Player'] = df['Player'].map(lambda x: x.replace('*','')) 
#df = pd.concat([df.iloc[:,:4].copy(), df.iloc[:,5:].copy().astype('str').round(2)], axis=1)
# x = df.iloc[:,5:].copy()
# x = x.applymap(lambda x: round(x,3))
# df = pd.concat([df.iloc[:,:5].copy() ,x], axis=1) 

In [ ]:
c = ['Player','MP','GS','TRB','PTS','STL%','BLK%','Offensive Creation','rTS%','Offensive Load']
ps_c =['Player'] + list(map(lambda feature: 'Ps' + feature,c[1:]))
c

**GET BOX SCORE DATA FOR EACH PLAYOFF GAME FROM 1991 - 1999**

In [ ]:
'''
team_mapping = {'Atlanta Hawks': 'ATL', 'Brooklyn Nets': 'BRK', 'Boston Celtics' : 'BOS', 'Charlotte Bobcats':'CHA','Charlotte Hornets': 'CHH', 'Chicago Bulls': 'CHI', 
                'Cleveland Cavaliers': 'CLE', 'Dallas Mavericks': 'DAL', 'Denver Nuggets': 'DEN', 'Detroit Pistons': 'DET', 'Golden State Warriors': 'GSW',
                'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND', 'Los Angeles Clippers': 'LAC', 'Los Angeles Lakers': 'LAL', 'Memphis Grizzlies': 'MEM',
                'Miami Heat': 'MIA', 'Milwaukee Bucks': 'MIL', 'Minnesota Timberwolves': 'MIN', 'New Orleans Hornets': 'NOH', 'New Orleans Pelicans': 'NOP', 'New Jersey Nets': 'NJN',
                'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC', 'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO', 
                'Portland Trail Blazers': 'POR','Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS', 'Seattle SuperSonics': 'SEA', 'Toronto Raptors': 'TOR',
                'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS'}
'''
day_dict = {1991:{4:list(range(25,31)), 5:list(range(1,32)), 6:list(range(1,13))}, 1992:{4:list(range(23,31)), 5:list(range(1,32)), 6:list(range(1,15))}, 
            1993:{4:list(range(29,31)), 5:list(range(1,32)), 6:list(range(1,21))}, 1994:{4:list(range(28,31)), 5:list(range(1,32)), 6:list(range(1,23))},
            1995:{4:list(range(27,31)), 5:list(range(1,32)), 6:list(range(1,15))}, 1996:{4:list(range(25,31)), 5:list(range(1,32)), 6:list(range(1,17))},
            1997:{4:list(range(24,31)), 5:list(range(1,32)), 6:list(range(1,14))}, 1998:{4:list(range(23,31)), 5:list(range(1,32)), 6:list(range(1,15))},
            1999:{4:list(range(19,31)), 5:list(range(1,32)), 6:list(range(1,26))}}

url = 'https://www.basketball-reference.com/boxscores/?month={m}&day={d}&year={y}'
html = urlopen(url)
soup = BeautifulSoup(html)
arrayofscores = []

for year in day_dict:
  print(year)
  for month in day_dict[year]:
    print(month)
    for day in day_dict[year][month]:
      bs_url = url.format(m = month,d=day,y=year)
      html = urlopen(bs_url)
      soup = BeautifulSoup(html)
      boxscores = soup.find_all("td", {"class": "right gamelink"})
      for boxscore in boxscores:
        boxscore_url = 'https://www.basketball-reference.com' + boxscore.a['href']
        print(boxscore_url)
        html = urlopen(boxscore_url)
        soup = BeautifulSoup(html)


        scores = {}


        header = soup.find('h1').getText()

        teams = header.split(' at ')
        at = teams[0].rstrip()
        scores['AT Name'] = at 

        print(teams)
        teams = teams[1].split('Box')
        ht = teams[0].rstrip().lstrip()
        scores['HT Name'] = ht

        score = soup.find_all(class_ = 'score')
        ats = score[0].getText()
        scores['AT Score'] = ats 
        hts = score[1].getText()
        scores['HT Score'] = hts 

        if hts > ats:
          scores['Home Win'] = 1
        
        if ats > hts:
          scores['Home Win'] = 0
        scores['Year'] = year
        scores['Month'] = month
        scores['Day'] = day

        scores_at_record = score[0].next.next.next.next
        scores_at_wl =  scores_at_record.split('-')
        scores['PS AT W/L Percentage'] = int(scores_at_wl[0])/(int(scores_at_wl[0])+int(scores_at_wl[1]))

        scores_ht_record = score[1].next.next.next.next
        scores_ht_wl =  scores_ht_record.split('-')
        scores['PS HT W/L Percentage'] = int(scores_ht_wl[0])/(int(scores_ht_wl[0])+int(scores_ht_wl[1]))


        team_url = 'https://www.basketball-reference.com/teams/{t}/{y}.html'

        team_urls = soup.find_all(class_ = 'scores')
        at_url_suffix = team_urls[0].previous_sibling.previous_sibling.find('strong').a['href']
        at_abbrev = at_url_suffix.split('/')[2]
        at_url = 'https://www.basketball-reference.com' + at_url_suffix
        at_html = urlopen(at_url)
        at_soup = BeautifulSoup(at_html)

        text = 'Off Rtg'

        at_ortg = at_soup.find(lambda tag: tag.name == "strong" and text in tag.text).next_sibling.split()[1]

        scores['AT Off Rtg'] = at_ortg

        text = 'Def Rtg'

        at_drtg = at_soup.find(lambda tag: tag.name == "strong" and text in tag.text).next_sibling.split()[1]
        scores['AT Def Rtg'] = at_drtg

        text = 'Record:'
        at_record = at_soup.find(lambda tag: tag.name == "strong" and text in tag.text).next_sibling.split()[0].split(',')[0].split('-')
        scores['AT Wins'] = int(at_record[0])
        scores['At Losses']= int(at_record[1])
        at_record = int(at_record[0])/(int(at_record[0]) + int(at_record[1]))
        scores['AT W/L Percentage'] = at_record

        

        team_urls = soup.find_all(class_ = 'scores')
        ht_url_suffix = team_urls[1].previous_sibling.previous_sibling.find('strong').a['href']
        ht_abbrev = ht_url_suffix.split('/')[2]
        ht_url = 'https://www.basketball-reference.com' + ht_url_suffix
        ht_html = urlopen(ht_url)
        ht_soup = BeautifulSoup(ht_html)

        text = 'Off Rtg'

        ht_ortg = ht_soup.find(lambda tag: tag.name == "strong" and text in tag.text).next_sibling.split()[1]
        scores['HT Off Rtg'] = ht_ortg

        text = 'Def Rtg'

        ht_drtg = ht_soup.find(lambda tag: tag.name == "strong" and text in tag.text).next_sibling.split()[1]
        scores['HT Def Rtg'] = ht_drtg

        text = 'Record:'
        ht_record = ht_soup.find(lambda tag: tag.name == "strong" and text in tag.text).next_sibling.split()[0].split(',')[0].split('-')
        scores['HT Wins'] = int(ht_record[0])
        scores['HT Losses'] = int(ht_record[1])
        ht_record = int(ht_record[0])/(int(ht_record[0]) + int(ht_record[1]))
        scores['HT W/L Percentage'] = ht_record

         

        at_players_bs = soup.find_all('table')[0].find_all('tbody')[0].find_all(class_ = 'left ')
        ht_players = pd.DataFrame()


        ht_players_bs = soup.find_all('table')[3].find_all('tbody')[0].find_all(class_ = 'left ')

        if year >= 1997:
          ht_players_bs = soup.find_all('table')[8].find_all('tbody')[0].find_all(class_ = 'left ')

        at_players = pd.DataFrame()
        ht_p = pd.DataFrame()
        ht_ps_p = pd.DataFrame()
        at_p = pd.DataFrame()
        at_ps_p = pd.DataFrame()

        for player in ht_players_bs:
          p = df[(df['Player'] == player.get_text()) & (df['Tm'] == ht_abbrev) &(df['Year'] == year)][c]
          ps_p = df[(df['Player'] == player.get_text()) & (df['PsMP'] != 0) &(df['Year'] == (year -1))][ps_c]
          if ps_p.empty:
            ps_p = pd.DataFrame(data=[[player.get_text(),0,0,0,0,0,0,0,0,0]],columns=ps_c)

          ht_ps_p = ht_ps_p.append(ps_p)
          ht_p = ht_p.append(p)
          # scores = {**scores,**ht_p}
          ht_players_int = pd.merge(ht_p,ht_ps_p, on=['Player'],how = 'outer').fillna(0).sort_values(by=['MP'],ascending=False).iloc[:8]

        ht_players = ht_players.append(ht_players_int)
        print(ht_players)
        ht_players = ht_players.iloc[:,1:]

        for player in at_players_bs:
          p = df[(df['Player'] == player.get_text()) & (df['Tm'] == at_abbrev) &(df['Year'] == year)][c]
          ps_p = df[(df['Player'] == player.get_text()) & (df['PsMP'] != 0) &(df['Year'] == (year -1))][ps_c]
          
          if ps_p.empty:
            ps_p = pd.DataFrame(data=[[player.get_text(),0,0,0,0,0,0,0,0,0]],columns=ps_c)

          
          at_ps_p = at_ps_p.append(ps_p)
          at_p = at_p.append(p)
          at_players_int = pd.merge(at_p,at_ps_p, on=['Player'],how = 'outer').fillna(0).sort_values(by=['MP'],ascending=False).iloc[:8]

        at_players = at_players.append(at_players_int)
        print(at_players)
        at_players = at_players.iloc[:,1:]


        for i in range(len(ht_players)):
           t_ht_players = ht_players.copy()
           t_c = t_ht_players.columns
           t_c = list(map(lambda a: a + '_ht' + str(i),t_c))
           t_ht_players.columns = t_c
           scores = {**scores,**t_ht_players.iloc[i].to_dict()}

        for i in range(len(at_players)):
           t_at_players = at_players.copy()
           t_c = t_at_players.columns
           t_c = list(map(lambda a: a + '_at' + str(i),t_c))
           t_at_players.columns = t_c
           scores = {**scores,**t_at_players.iloc[i].to_dict()}
        print(scores)
          
        arrayofscores.append(scores)

dataframe = pd.DataFrame(arrayofscores)
dataframe.to_csv('scores1991.csv') 
files.download('scores1991.csv')
 

Streaming output truncated to the last 5000 lines.
          Player    MP  GS  ...  PsOffensive Creation  PsrTS%    PsOffensive Load
0  Allen Iverson  1990  48  ...                   0.0     0.0                 0.0
1      Eric Snow  1716  48  ...                   0.0     0.0                 0.0
2   Theo Ratliff  1627  50  ...                   0.0     0.0                 0.0
3    Matt Geiger  1540  40  ...                 0.041  -0.362 20.9289999999999985
5   Larry Hughes   988   1  ...                   0.0     0.0                 0.0
7    Aaron McKie   959   4  ...                   0.0     0.0                 0.0
4   Harvey Grant   798  10  ...                   0.0     0.0                 0.0
6    Tyrone Hill   587   6  ...                   0.0     0.0                 0.0

[8 rows x 19 columns]
             Player    MP  ...  PsrTS%     PsOffensive Load
0     Allan Houston  1815  ...  -0.003  38.9909999999999997
1     Larry Johnson  1639  ...   0.025  31.5189999999999984
3      C

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

GET BOX SCORE DATA FOR EACH GAME IN THE 2021 PLAYOFFS 

In [ ]:

day_dict = {2021:{5:list(range(22,32)), 6:list(range(1,32)), 7:list(range(1,32))}}

url = 'https://www.basketball-reference.com/boxscores/?month={m}&day={d}&year={y}'
html = urlopen(url)
soup = BeautifulSoup(html)
arrayofscores = []

for year in day_dict:
  print(year)
  for month in day_dict[year]:
    print(month)
    for day in day_dict[year][month]:
      bs_url = url.format(m = month,d=day,y=year)
      html = urlopen(bs_url)
      soup = BeautifulSoup(html)
      boxscores = soup.find_all("td", {"class": "right gamelink"})
      for boxscore in boxscores:
        boxscore_url = 'https://www.basketball-reference.com' + boxscore.a['href']
        print(boxscore_url)
        html = urlopen(boxscore_url)
        soup = BeautifulSoup(html)


        scores = {}


        header = soup.find('h1').getText()

        teams = header.split(' at ')
        at = teams[0].rstrip()
        scores['AT Name'] = at 

        print(teams)
        teams = teams[1].split('Box')
        ht = teams[0].rstrip().lstrip()
        scores['HT Name'] = ht

        score = soup.find_all(class_ = 'score')
        ats = score[0].getText()
        scores['AT Score'] = ats 
        hts = score[1].getText()
        scores['HT Score'] = hts 

        scores['Year'] = year
        scores['Month'] = month
        scores['Day'] = day

        scores_at_record = score[0].next.next.next.next
        scores_at_wl =  scores_at_record.split('-')
        scores['PS AT W/L Percentage'] = int(scores_at_wl[0])/(int(scores_at_wl[0])+int(scores_at_wl[1]))
        

        scores_ht_record = score[1].next.next.next.next
        scores_ht_wl =  scores_ht_record.split('-')
        scores['PS HT W/L Percentage'] = int(scores_ht_wl[0])/(int(scores_ht_wl[0])+int(scores_ht_wl[1]))
        

        team_url = 'https://www.basketball-reference.com/teams/{t}/{y}.html'

        team_urls = soup.find_all(class_ = 'scores')
        at_url_suffix = team_urls[0].previous_sibling.previous_sibling.find('strong').a['href']
        at_abbrev = at_url_suffix.split('/')[2]
        at_url = 'https://www.basketball-reference.com' + at_url_suffix
        at_html = urlopen(at_url)
        at_soup = BeautifulSoup(at_html)

        text = 'Off Rtg'

        at_ortg = at_soup.find(lambda tag: tag.name == "strong" and text in tag.text).next_sibling.split()[1]

        scores['AT Off Rtg'] = at_ortg

        text = 'Def Rtg'

        at_drtg = at_soup.find(lambda tag: tag.name == "strong" and text in tag.text).next_sibling.split()[1]
        scores['AT Def Rtg'] = at_drtg

        text = 'Record:'
        at_record = at_soup.find(lambda tag: tag.name == "strong" and text in tag.text).next_sibling.split()[0].split(',')[0].split('-')
        at_record = int(at_record[0])/(int(at_record[0]) + int(at_record[1]))
        scores['AT W/L Percentage'] = at_record


        team_urls = soup.find_all(class_ = 'scores')
        ht_url_suffix = team_urls[1].previous_sibling.previous_sibling.find('strong').a['href']
        ht_abbrev = ht_url_suffix.split('/')[2]
        ht_url = 'https://www.basketball-reference.com' + ht_url_suffix
        ht_html = urlopen(ht_url)
        ht_soup = BeautifulSoup(ht_html)

        text = 'Off Rtg'

        ht_ortg = ht_soup.find(lambda tag: tag.name == "strong" and text in tag.text).next_sibling.split()[1]
        scores['HT Off Rtg'] = ht_ortg

        text = 'Def Rtg'

        ht_drtg = ht_soup.find(lambda tag: tag.name == "strong" and text in tag.text).next_sibling.split()[1]
        scores['HT Def Rtg'] = ht_drtg

        text = 'Record:'
        ht_record = ht_soup.find(lambda tag: tag.name == "strong" and text in tag.text).next_sibling.split()[0].split(',')[0].split('-')
        ht_record = int(ht_record[0])/(int(ht_record[0]) + int(ht_record[1]))
        scores['HT W/L Percentage'] = ht_record
        

        at_players_bs = soup.find_all('table')[0].find_all('tbody')[0].find_all(class_ = 'left ')
        ht_players = pd.DataFrame()
        ht_players_bs = soup.find_all('table')[8].find_all('tbody')[0].find_all(class_ = 'left ')
        at_players = pd.DataFrame()
        ht_p = pd.DataFrame()
        ht_ps_p = pd.DataFrame()
        at_p = pd.DataFrame()
        at_ps_p = pd.DataFrame()

        for player in ht_players_bs:
          p = df[(df['Player'] == player.get_text()) & (df['Tm'] == ht_abbrev) &(df['Year'] == year)][c]
          ps_p = df[(df['Player'] == player.get_text()) & (df['PsMP'] != 0) &(df['Year'] == (year -1))][ps_c]
          if ps_p.empty:
            ps_p = pd.DataFrame(data=[[player.get_text(),0,0,0,0,0,0,0,0,0]],columns=ps_c)

          ht_ps_p = ht_ps_p.append(ps_p)
          ht_p = ht_p.append(p)
          # scores = {**scores,**ht_p}
          ht_players_int = pd.merge(ht_p,ht_ps_p, on=['Player'],how = 'outer').fillna(0).sort_values(by=['MP'],ascending=False).iloc[:8]

        ht_players = ht_players.append(ht_players_int)
        print(ht_players)
        ht_players = ht_players.iloc[:,1:]

        for player in at_players_bs:
          p = df[(df['Player'] == player.get_text()) & (df['Tm'] == at_abbrev) &(df['Year'] == year)][c]
          ps_p = df[(df['Player'] == player.get_text()) & (df['PsMP'] != 0) &(df['Year'] == (year -1))][ps_c]
          
          if ps_p.empty:
            ps_p = pd.DataFrame(data=[[player.get_text(),0,0,0,0,0,0,0,0,0]],columns=ps_c)

          
          at_ps_p = at_ps_p.append(ps_p)
          at_p = at_p.append(p)
          at_players_int = pd.merge(at_p,at_ps_p, on=['Player'],how = 'outer').fillna(0).sort_values(by=['MP'],ascending=False).iloc[:8]

        at_players = at_players.append(at_players_int)
        print(at_players)
        at_players = at_players.iloc[:,1:]


        for i in range(0,8):
           t_ht_players = ht_players.copy()
           t_c = t_ht_players.columns
           t_c = list(map(lambda a: a + '_ht' + str(i),t_c))
           t_ht_players.columns = t_c
           scores = {**scores,**t_ht_players.iloc[i].to_dict()}

        for i in range(0,8):
           t_at_players = at_players.copy()
           t_c = t_at_players.columns
           t_c = list(map(lambda a: a + '_at' + str(i),t_c))
           t_at_players.columns = t_c
           scores = {**scores,**t_at_players.iloc[i].to_dict()}
        print(scores)
          
        arrayofscores.append(scores)

dataframe = pd.DataFrame(arrayofscores)
dataframe.to_csv('2021scores.csv') 
files.download('2021scores.csv')
 

**GET ELO RATINGS FOR EACH TEAM FROM 1999 - 2020**

In [ ]:
day_dict = {1999:{11:range(2,31),12:range(1,32)}, 2000:{1:range(1,32),2:range(1,30),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,20),10:[31],11:range(1,31),12:range(1,32)}, 2001:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,16), 10:[30,31],11:range(1,31),12:range(1,32)},
              2002:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,13),10:range(29,32),11:range(1,31),12:range(1,32)}, 2003:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,16),10:range(28,32),11:range(1,31),12:range(1,32)},
              2004:{1:range(1,32),2:range(1,30),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,16), 11:range(2,31),12:range(1,32)}, 2005:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,24),11:range(1,31),12:range(1,32)},2006:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,21),10:range(31,32),11:range(1,31),12:range(1,32)},
              2007:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,15),10:range(30,32),11:range(1,31),12:range(1,32)}, 2008:{1:range(1,32),2:range(1,30),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,18),10:range(28,32),11:range(1,31),12:range(1,32)}, 2009:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,15),10:range(27,32),11:range(1,31),12:range(1,32)},
              2010:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,18),10:range(26,32),11:range(1,31),12:range(1,32)}, 2011:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,13),12:range(25,32)}, 2012:{1:range(1,32),2:range(1,30),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,22),10:range(30,32),11:range(1,31),12:range(1,32)},
              2013:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,21),10:range(29,32),11:range(1,31),12:range(1,32)}, 2014:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,16),10:range(28,32),11:range(1,31),12:range(1,32)}, 2015:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,17),10:range(27,32),11:range(1,31),12:range(1,32)}, 
              2016:{1:range(1,32),2:range(1,30),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,20),10:range(25,32),11:range(1,31),12:range(1,32)}, 2017:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,13),10:range(17,32),11:range(1,31),12:range(1,32)}, 2018:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,9),10:range(16,32),11:range(1,31),12:range(1,32)},
              2019:{1:range(1,32),2:range(1,29),3:range(1,32),4:range(1,31),5:range(1,32),6:range(1,14),10:range(22,32),11:range(1,31),12:range(1,32)}, 2020:{1:range(1,32),2:range(1,30),3:range(1,12),7:range(30,32),8:range(1,32),9:range(1,31),10:range(1,11)}}
              
new_season_dict = {2000:{10:31},2001:{10:30},2002:{10:29},2003:{10:28},2004:{11:2},2005:{11:1},2006:{10:31},2007:{10:30},
                   2008:{10:28},2009:{10:27},2010:{10:26},2011:{12:25},2012:{10:30},2013:{10:29},2014:{10:28},2015:{10:27},
                   2016:{10:25},2017:{10:17},2018:{10:16},2019:{10:22},2020:{12:22}}
team_mapping = {'CHH':'CHA', 'CHO':'CHA','SEA':'OKC', 'NJN': 'BRK', 'VAN': 'MEM','NOH':'NOP','NOK':'NOP'}

base_elortgs = {'ATL':1550, 'BOS':1467, 'BRK':1460, 'CHA': 1547, 'CHI': 1393, 'CLE': 1467, 'DAL': 1443, 'DEN': 1338, 'DET': 1570, 'GSW': 1432,
                'HOU': 1507, 'IND': 1616, 'LAC': 1348, 'LAL': 1559, 'MEM': 1290, 'MIA': 1566, 'MIL': 1509, 'MIN': 1472, 'NOP': 1300, 'NYK': 1592,
                'OKC': 1484, 'ORL': 1540, 'PHI': 1542, 'PHO': 1541, 'POR': 1569, 'SAC': 1489, 'SAS': 1693, 'TOR': 1455, 'UTA': 1593, 'WAS': 1443}
# elo_df = pd.DataFrame([base_elortgs])
# elo_df['Year'] = 1999
# elo_df['Month'] = 11
# elo_df['Day'] = 1

# game_before_elo_df = pd.DataFrame([base_elortgs])
# game_before_elo_df['Year'] = 1999
# game_before_elo_df['Month'] = 11
# game_before_elo_df['Day'] = 1


url = 'https://www.basketball-reference.com/boxscores/?month={m}&day={d}&year={y}'
html = urlopen(url)
soup = BeautifulSoup(html)

for year in day_dict:
  for month in day_dict[year]:
    print(month)

    for day in day_dict[year][month]:
      print(day)
      bs_url = url.format(m = month,d=day,y=year)
      html = urlopen(bs_url)
      soup = BeautifulSoup(html)
      boxscores = soup.find_all("td", {"class": "right gamelink"})
      if boxscores != []:
        elo_df.iloc[-1:30]
        elo_df = elo_df.append(elo_df.iloc[-1,:])
        elo_df = elo_df.reset_index(drop=True)
        elo_df['Day'].iloc[-1] = day
        elo_df['Month'].iloc[-1] = month
        elo_df['Year'].iloc[-1] = year
        
        if year in new_season_dict:
          if month not in new_season_dict[year]:
            game_before_elo_df = game_before_elo_df.append(elo_df.iloc[-1,:])
            game_before_elo_df = game_before_elo_df.reset_index(drop=True)
            print('A')
            print(game_before_elo_df.iloc[-1])
          if month in new_season_dict[year]:
            print('B')
            if day != new_season_dict[year][month]:
              game_before_elo_df = game_before_elo_df.append(elo_df.iloc[-1,:])
              game_before_elo_df = game_before_elo_df.reset_index(drop=True)
              print('B1')

        else:
          game_before_elo_df = game_before_elo_df.append(elo_df.iloc[-1,:])
          game_before_elo_df = game_before_elo_df.reset_index(drop=True)
      
      if year in new_season_dict:
        if month in new_season_dict[year]:
          print('C')
          if day == new_season_dict[year][month]:
            print('C1')
            elo_df.iloc[-1,:30] = 0.75 * elo_df.iloc[-1,:30] + 0.25 * 1505
            game_before_elo_df = game_before_elo_df.append(elo_df.iloc[-1,:])
            game_before_elo_df = game_before_elo_df.reset_index(drop=True)
            print(elo_df.iloc[-1])
            if year == 2002:
              elo_df['NOP'].iloc[-1] = 1541
            if year == 2004:
              elo_df['CHA'].iloc[-1] = 1300
            print('Im here')

      for boxscore in boxscores:
        boxscore_url = 'https://www.basketball-reference.com' + boxscore.a['href']
        print(boxscore_url)
        html = urlopen(boxscore_url)
        soup = BeautifulSoup(html)


        header = soup.find('h1').getText()

        teams = header.split(' at ')
        at = teams[0].rstrip()

        print(teams)
        teams = teams[1].split('Box')
        ht = teams[0].rstrip().lstrip()

        team_urls = soup.find_all(class_ = 'scores')
        ht_url_suffix = team_urls[1].previous_sibling.previous_sibling.find('strong').a['href']
        ht_abbrev = ht_url_suffix.split('/')[2]

        at_url_suffix = team_urls[0].previous_sibling.previous_sibling.find('strong').a['href']
        at_abbrev = at_url_suffix.split('/')[2]

        score = soup.find_all(class_ = 'score')
        ats = int(score[0].getText())
        hts = int(score[1].getText())

        if at_abbrev in team_mapping:
          at_abbrev = team_mapping[at_abbrev]
        
        if ht_abbrev in team_mapping:
          ht_abbrev = team_mapping[ht_abbrev]


        ht_elo = elo_df[ht_abbrev].iloc[-1] + 100
        at_elo = elo_df[at_abbrev].iloc[-1]
        ht_elo_int = float(1/(1+10**((at_elo - ht_elo)/400)))

        at_elo_int = float(1/(1+10**((ht_elo - at_elo)/400)))
        

        if int(hts) > int(ats):
          k = float(20 * ((hts - ats + 3) ** (0.8))/(7.5 + 0.006*(ht_elo - at_elo)))
          at_elo = k*(-at_elo_int) + at_elo
          ht_elo = k*(1-ht_elo_int) + ht_elo

        if int(ats) > int(hts):

          k = float(20 * ((ats - hts + 3) ** (0.8))/(7.5 + 0.006*(at_elo - ht_elo)))
          at_elo = k*(1-at_elo_int) + at_elo
          ht_elo = k*(-ht_elo_int) + ht_elo

        ht_elo = ht_elo - 100
        elo_df[ht_abbrev].iloc[-1] = float(ht_elo) 
        elo_df[at_abbrev].iloc[-1] = float(at_elo)

elo_df.to_csv('elo_df.csv') 
files.download('elo_df.csv')

game_before_elo_df.to_csv('game_before_elo_df.csv') 
files.download('game_before_elo_df.csv')
